读取数据

In [28]:
import pandas as pd

# 只需要这些features
features = ['accommodates','bedrooms','bathrooms','beds',
            'price','minimum_nights','maximum_nights','number_of_reviews']
# 读取数据
dc_listings = pd.read_csv('listings.csv')[features]

print(dc_listings.shape)
dc_listings.head()

(3723, 8)


,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
0,4,1.0,1.0,2.0,$160.00,1,1125,0
1,6,3.0,3.0,3.0,$350.00,2,30,65
2,1,1.0,2.0,1.0,$50.00,2,1125,1
3,2,1.0,1.0,1.0,$95.00,1,1125,0
4,4,1.0,1.0,1.0,$50.00,7,1125,0


数据特征：

* accommodates: 可以容纳的旅客
* bedrooms: 卧室的数量
* bathrooms: 厕所的数量
* beds: 床的数量
* price: 每晚的费用
* minimum_nights: 客人最少租了几天
* maximum_nights: 客人最多租了几天
* number_of_reviews: 评论的数量

In [29]:
# 按照房间数量绝对值差异算距离
import numpy as np

our_acc_value = 3

dc_listings['distance'] = np.abs(dc_listings.accommodates - our_acc_value)
dc_listings.distance.value_counts().sort_index()

0      461
1     2294
2      503
3      279
4       35
5       73
6       17
7       22
8        7
9       12
10       2
11       4
12       6
13       8
Name: distance, dtype: int64

In [30]:
# sample操作可以得到洗牌后的数据
dc_listings = dc_listings.sample(frac=1, random_state=0)
dc_listings = dc_listings.sort_values('distance')
dc_listings.price.head()

2645     $75.00
2825    $120.00
2145     $90.00
2541     $50.00
3349    $105.00
Name: price, dtype: object

In [31]:
# 字符串转化成数字并算平均价格
dc_listings['price'] = dc_listings.price.str.replace('\$|,','',regex=True).astype(float)

mean_price = dc_listings.price.iloc[:5].mean()
mean_price

88.0

首先制定好训练集和测试集

In [32]:
dc_listings.drop('distance', axis=1)

train_df = dc_listings.copy().iloc[:2792]
test_df = dc_listings.copy().iloc[2792:]

基于单变量预测价格

In [40]:
def predict_price(new_listing_value, feature_column):
    temp_df = train_df
    temp_df['distance'] = np.abs(dc_listings[feature_column] - new_listing_value)
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    return(predicted_price)

In [41]:
test_df.accommodates[:5]

2850    1
2279    1
2771    5
910     5
2434    5
Name: accommodates, dtype: int64

In [42]:
test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column='accommodates')

root mean squared error (RMSE) 均方根误差
<img src="8.png" style="width:700px;height:100px;float:left">

In [44]:
test_df['squared_error'] = (test_df['predicted_price'] - 
                           test_df['price'])**2
mse = test_df['squared_error'].mean()
rmse = mse ** (1/2)
rmse

212.98927967051543

In [45]:
for feature in ['accommodates','bedrooms','bathrooms','number_of_reviews']:
    #test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column=feature)
    test_df['predicted_price'] = test_df[feature].apply(predict_price,feature_column=feature)
    test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
    mse = test_df['squared_error'].mean()
    rmse = mse ** (1/2)
    print("RMSE for the {} column: {}".format(feature,rmse))

RMSE for the accommodates column: 212.98927967051543
RMSE for the bedrooms column: 199.80935328065033
RMSE for the bathrooms column: 230.24716705684227
RMSE for the number_of_reviews column: 235.91327066995507


In [47]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
features = ['accommodates','bedrooms','bathrooms','beds','price','minimum_nights','maximum_nights','number_of_reviews']
dc_listings = pd.read_csv('listings.csv')[features]
dc_listings['price'] = dc_listings.price.str.replace('\$|,','',regex=True).astype(float)
# 过滤缺失值
dc_listings = dc_listings.dropna()
dc_listings[features] = StandardScaler().fit_transform(dc_listings[features])
normalized_listings = dc_listings
print(dc_listings.shape)
normalized_listings.head()

(3671, 8)


,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
0,0.401420,-0.249501,-0.439211,0.297386,0.081119,-0.341421,-0.016575,-0.516779
1,1.399466,2.129508,2.969551,1.141704,1.462622,-0.065047,-0.016606,1.706767
2,-1.095648,-0.249501,1.265170,-0.546933,-0.718699,-0.065047,-0.016575,-0.482571
3,-0.596625,-0.249501,-0.439211,-0.546933,-0.391501,-0.341421,-0.016575,-0.516779
4,0.401420,-0.249501,-0.439211,-0.546933,-0.718699,1.316824,-0.016575,-0.516779


In [48]:
norm_train_df = normalized_listings.copy().iloc[0:2792]
norm_test_df = normalized_listings.copy().iloc[2792:]

scipy中已有现成的距离的计算工具了

In [50]:
from scipy.spatial import distance

first_listing = normalized_listings.iloc[0][['accommodates','bathrooms']]
fifth_listing = normalized_listings.iloc[20][['accommodates','bathrooms']]
first_fifth_distance = distance.euclidean(first_listing, fifth_listing)
first_fifth_distance

3.723019604017032

多变量KNN模型

In [51]:
def predict_price_multivariate(new_listing_value,feature_columns):
    temp_df = norm_train_df
    temp_df['distance'] = distance.cdist(temp_df[feature_columns],[new_listing_value[feature_columns]])
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    return(predicted_price)

cols = ['accommodates', 'bathrooms']
norm_test_df['predicted_price'] = norm_test_df[cols].apply(predict_price_multivariate,feature_columns=cols,axis=1)    
norm_test_df['squared_error'] = (norm_test_df['predicted_price'] - norm_test_df['price'])**(2)
mse = norm_test_df['squared_error'].mean()
rmse = mse ** (1/2)
print(rmse)

0.7894063922577537


使用Sklearn来完成KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor